# Creating a RO-crate entry and serializing it in JSON-LD
https://researchobject.github.io/ro-crate/

In [1]:
from rdflib import *
from datetime import datetime

schema = Namespace("http://schema.org/")

# Writing RDF triples to populate a minimal RO-crate

In [2]:
graph = ConjunctiveGraph()
#graph.bind('foaf', 'http://xmlns.com/foaf/0.1/')
graph.load('https://researchobject.github.io/ro-crate/0.2/context.json', format='json-ld')

# person information
graph.add( (URIRef('https://orcid.org/0000-0002-3597-8557'), RDF.type, schema.Person) )

# contact information
graph.add( (URIRef('alban.gaignard@univ-nantes.fr'), RDF.type, schema.ContactPoint) )
graph.add((URIRef('alban.gaignard@univ-nantes.fr'), schema.contactType, Literal('Developer')) )
graph.add( (URIRef('alban.gaignard@univ-nantes.fr'), schema.name, Literal('Alban Gaignard')) )
graph.add( (URIRef('alban.gaignard@univ-nantes.fr'), schema.email, Literal('alban.gaignard@univ-nantes.fr', datatype=XSD.string)) )
graph.add( (URIRef('alban.gaignard@univ-nantes.fr'), schema.url, Literal('https://orcid.org/0000-0002-3597-8557')) )

# root metadata
graph.add( (URIRef('ro-crate-metadata.jsonld'), RDF.type, schema.CreativeWork) )
graph.add( (URIRef('ro-crate-metadata.jsonld'), schema.identifier, Literal('ro-crate-metadata.jsonld')) )
graph.add( (URIRef('ro-crate-metadata.jsonld'), schema.about, URIRef('./')) )

# Dataset metadata with reference to files
graph.add( (URIRef('./'), RDF.type, schema.Dataset) )
graph.add( (URIRef('./'), schema.name, Literal("workflow outputs")) )
graph.add( (URIRef('./'), schema.datePublished, Literal(datetime.now().isoformat())) )
graph.add( (URIRef('./'), schema.author, URIRef('https://orcid.org/0000-0002-3597-8557')) )
graph.add( (URIRef('./'), schema.contactPoint, URIRef('alban.gaignard@univ-nantes.fr')) )
graph.add( (URIRef('./'), schema.description, Literal("this is the description of the workflow description, this is the description of the workflow description, this is the description of the workflow description")) )
graph.add( (URIRef('./'), schema.license, Literal("MIT?")) )
graph.add( (URIRef('./'), schema.hasPart, (URIRef('./data/provenance.ttl'))) ) 

# Files metadata 
graph.add( (URIRef('./data/provenance.ttl'), RDF.type, schema.MediaObject) ) 

print(graph.serialize(format='turtle').decode())
#print(graph.serialize(format='json-ld').decode())

@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix cc: <http://creativecommons.org/ns#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix frapo: <http://purl.org/cerif/frapo/> .
@prefix pav: <http://purl.org/pav/> .
@prefix pcdm: <http://pcdm.org/models#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfa: <http://www.w3.org/ns/rdfa#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rel: <https://www.w3.org/ns/iana/link-relations/relation#> .
@prefix roterms: <http://purl.org/ro/roterms#> .
@prefix schema: <http://schema.org/> .
@prefix wf4ever: <http://purl.org/ro/wf4ever#> .
@prefix wfdesc: <http://purl.org/ro/wfdesc#> .
@prefix wfprov: <http://purl.org/ro/wfprov#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<ro-crate-metadata.jsonld> a schema:CreativeWork ;
    schema:about <./> ;


# Warping these triples into Python objects

In [3]:
import requests 
import json

class RO_crate_abstract:
    """
    An abstract RO-crate class to share common attributes and methods. 
    """

    def __init__(self, uri):
        self.uri = uri
        self.graph = ConjunctiveGraph()
        
    def get_uri(self):
        return self.uri
    
    def print(self):
        print(self.graph.serialize(format='turtle').decode())
    
    def serialize_jsonld(self):
        res = requests.get('https://w3id.org/ro/crate/1.0/context')
        ctx = json.loads(res.text)['@context']

        jsonld = self.graph.serialize(format='json-ld', context=ctx)
        print(jsonld.decode())
        self.graph.serialize(destination='ro-crate-metadata.jsonld', format='json-ld', context=ctx)
        
    def add_has_part(self, other_ro_crate):
        self.graph = self.graph + other_ro_crate.graph
        #TODO add has_part property 
        self.graph.add( (URIRef(self.get_uri()), schema.hasPart, URIRef(other_ro_crate.get_uri())) )
        
        

class RO_crate_Root(RO_crate_abstract):
    """
    The root RO-crate. 
    """
    
    def __init__(self):
        RO_crate_abstract.__init__(self, uri='ro-crate-metadata.jsonld')
        self.graph.add( (URIRef('ro-crate-metadata.jsonld'), RDF.type, schema.CreativeWork) )
        self.graph.add( (URIRef('ro-crate-metadata.jsonld'), schema.identifier, Literal('ro-crate-metadata.jsonld')) )
        self.graph.add( (URIRef('ro-crate-metadata.jsonld'), schema.about, URIRef('./')) )


class RO_crate_Person(RO_crate_abstract):
    """
    A person RO-crate. 
    """
    
    def __init__(self, uri):
        RO_crate_abstract.__init__(self, uri)
        self.graph.add( (URIRef(uri), RDF.type, schema.Person) )
        

class RO_crate_Contact(RO_crate_Person):
    """
    A person RO-crate. 
    """
    
    def __init__(self, uri, name=None, email=None, ctype=None, url=None):
        RO_crate_Person.__init__(self, uri)
        self.graph.add( (URIRef(uri), RDF.type, schema.Person) )
        if name:
            self.graph.add( (URIRef(uri), schema.name, Literal(name)) )
        if email:
            self.graph.add( (URIRef(uri), schema.email, Literal(email, datatype=XSD.string)) )
        if ctype:
            self.graph.add( (URIRef(uri), schema.contactType, Literal(ctype)) )
        if url:
            self.graph.add( (URIRef(uri), schema.url, Literal(url)) )
    
    

In [62]:
# creating a root RO-crate        
root = RO_crate_Root()
root.print()

# creating a person RO-crate        
person = RO_crate_Person('https://orcid.org/0000-0002-3597-8557')
person.print()

# creating a contact RO-crate        
contact = RO_crate_Contact('https://orcid.org/0000-0002-3597-8557', name='Alban Gaignard', ctype='contributor')
contact.print()

# adding hasPart relation between RO-crates
root.add_has_part(contact)
root.print()

@prefix ns1: <http://schema.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<ro-crate-metadata.jsonld> a ns1:CreativeWork ;
    ns1:about <./> ;
    ns1:identifier "ro-crate-metadata.jsonld" .


@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://orcid.org/0000-0002-3597-8557> a <http://schema.org/Person> .


@prefix ns1: <http://schema.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://orcid.org/0000-0002-3597-8557> a ns1:Person ;
    ns1:cont

In [61]:
# serializing the output
root.serialize_jsonld()

{
  "@context": {
    "3DModel": "http://schema.org/3DModel",
    "@base": null,
    "@label": "http://www.w3.org/2000/01/rdf-schema#label",
    "AMRadioChannel": "http://schema.org/AMRadioChannel",
    "APIReference": "http://schema.org/APIReference",
    "Abdomen": "http://schema.org/Abdomen",
    "AboutPage": "http://schema.org/AboutPage",
    "AcceptAction": "http://schema.org/AcceptAction",
    "Accommodation": "http://schema.org/Accommodation",
    "AccountingService": "http://schema.org/AccountingService",
    "AchieveAction": "http://schema.org/AchieveAction",
    "Action": "http://schema.org/Action",
    "ActionAccessSpecification": "http://schema.org/ActionAccessSpecification",
    "ActionStatusType": "http://schema.org/ActionStatusType",
    "ActivateAction": "http://schema.org/ActivateAction",
    "ActiveActionStatus": "http://schema.org/ActiveActionStatus",
    "ActiveNotRecruiting": "http://schema.org/ActiveNotRecruiting",
    "AddAction": "http://schema.org/AddAction",
 